Creating a connection to the database and getting the data

In [221]:
import psycopg2
import matplotlib.pyplot as plt
import numpy as np

# connecting to database to retrieve pricing data
conn = psycopg2.connect("dbname=postgres user=postgres password=*********")
cursor = conn.cursor()
# getting accepted prices
cursor.execute('SELECT pay FROM pricing."driverAcceptanceData" WHERE accepted = 1')
accepted_list = cursor.fetchall()
# getting not accepted prices
cursor.execute('SELECT pay FROM pricing."driverAcceptanceData" WHERE accepted = 0')
refused_list = cursor.fetchall()
# getting all prices
cursor.execute('SELECT pay FROM pricing."driverAcceptanceData"')
price_list = cursor.fetchall()
# getting zeros and ones
cursor.execute('SELECT accepted FROM pricing."driverAcceptanceData"')
accepted = cursor.fetchall()

Running logistic regression on the data to estimate when drivers are going to accept or refuse offers

In [222]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

x = np.array(price_list).reshape(-1,1)
y = np.array(accepted).reshape(-1,1)

model = LogisticRegression(solver='liblinear',C=10.0, random_state=0).fit(x,y)
print(model.score(x,y))

price = 0
t = np.array([price]).reshape(-1,1)
while(model.predict(t)==0):
    price += 0.01
    t = np.array([price]).reshape(-1,1)
    
print('Price: $',price,' is the minimum pay for which the model predicts a drivers accepatnce')

0.833


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Running the model to create expected profits for the 12 month periods of the case

In [223]:
# calculating expected value of requested rides
from scipy.stats import poisson

possible_rides = 0
avg_past_rides = 1
quantity_tot_riders = 0
expected_rides = 0
month = 1
marginal_cost_ride = 24.86
marginal_revenue = 30
marginal_cost_acquired_user = 30
yearly_total_cost = 0
yearly_total_revenue = 0
quantity_riders_acquired = 1000

for month in (1,2,3,4,5,6,7,8,9,10,11,12):
    print('month ', month)
    print('new amount of customer acquired for this month is: ', quantity_riders_acquired)
    quantity_tot_riders += quantity_riders_acquired

    for possible_rides in (0,1,2,3,4,5):
        prob = poisson.pmf(k=possible_rides, mu=avg_past_rides)
        rides = prob * quantity_tot_riders * possible_rides
        expected_rides += round(rides)

    print('expected rides at lambda=:', avg_past_rides, ' is: ', expected_rides)
    quantity_user_churn = round(poisson.pmf(k=0, mu=avg_past_rides)*quantity_tot_riders)
    print('quantity of user leaving the platform: ', quantity_user_churn, ' a rate of: ', poisson.pmf(k=0, mu=avg_past_rides))

    avg_past_rides = round(expected_rides / quantity_tot_riders)
    print('new lambda: ', avg_past_rides)

    quantity_tot_riders -= quantity_user_churn
    print('new total amount of users: ', quantity_tot_riders)

    monthly_total_cost_acquired_user = quantity_riders_acquired * marginal_cost_acquired_user
    print('total cost acquiring users: ', round(monthly_total_cost_acquired_user))
    monthly_total_cost_rides = expected_rides * marginal_cost_ride
    print('total cost of drivers: ', round(monthly_total_cost_rides))
    monthly_total_cost = monthly_total_cost_acquired_user + monthly_total_cost_rides
    print('total cost: ', round(monthly_total_cost))
    monthly_total_revenue = expected_rides * marginal_revenue
    print('total revenue: ', round(monthly_total_revenue))
    monthly_profit = monthly_total_revenue - monthly_total_cost
    print('monthly profit: ', round(monthly_profit))
    yearly_total_cost += monthly_total_cost
    yearly_total_revenue += monthly_total_revenue 
   
    print('maximising profit rule: rides needs to be 6 times the quantity of new acquired riders')
    print('currently we have: ', (expected_rides/quantity_riders_acquired)) 
    print(' ')

print('yearly total revenue: $', round(yearly_total_revenue))
print('yearly total cost: $', round(yearly_total_cost))
yearly_total_profit = yearly_total_revenue - yearly_total_cost
print('yearly total profits: $', round(yearly_total_profit))

month  1
new amount of customer acquired for this month is:  1000
expected rides at lambda=: 1  is:  996
quantity of user leaving the platform:  368  a rate of:  0.36787944117144233
new lambda:  1
new total amount of users:  632
total cost acquiring users:  30000
total cost of drivers:  24761
total cost:  54761
total revenue:  29880
monthly profit:  -24881
maximising profit rule: rides needs to be 6 times the quantity of new acquired riders
currently we have:  0.996
 
month  2
new amount of customer acquired for this month is:  1000
expected rides at lambda=: 1  is:  2621
quantity of user leaving the platform:  600  a rate of:  0.36787944117144233
new lambda:  2
new total amount of users:  1032
total cost acquiring users:  30000
total cost of drivers:  65158
total cost:  95158
total revenue:  78630
monthly profit:  -16528
maximising profit rule: rides needs to be 6 times the quantity of new acquired riders
currently we have:  2.621
 
month  3
new amount of customer acquired for this mo